# [NTDS'18] tutorial 5: sparse matrices in scipy
[ntds'18]: https://github.com/mdeff/ntds_2018

[Eda Bayram](http://lts4.epfl.ch/bayram), [EPFL LTS4](http://lts4.epfl.ch)

## Ojective

This is a short tutorial on the `scipy.sparse` module. We will talk about:

1. What is sparsity?
2. Sparse matrix storage schemes
3. Linear operations on sparse matrices

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
import scipy.sparse.linalg
from scipy import linalg
import pandas as pd

## 1. Sparsity

Why do we need data structures for sparse matrices?

* Less memory usage
* More efficiency computations

Most real-world graphs / networks are sparse!

Let us create a random sparse matrix and analyze the sparsity.

In [ ]:
N = 250 
dummy = sparse.random(N, N, density=0.01)
density = dummy.nnz / N**2
print('Number of non-zeros: {}, density: {}'.format(dummy.nnz, density))

In [ ]:
plt.spy(dummy, markersize=1);

In [ ]:
print(dummy)

Let us convert the sparse array to some dense formats.

In [ ]:
type(dummy.A)

In [ ]:
type(dummy.toarray())

In [ ]:
type(dummy.todense())

## 2. Sparse matrix storage schemes

The `scipy.sparse` module provides several formats to store sparse matrices.
Each format has pros and cons, and some are better for some tasks, such as matrix construction, indexing, or linear operations.

### 2.1 List of lists format (LIL)

* Supports indexing, which cannot be done with other sparse matrix formats.
* Changing sparsity structure is efficient, e.g., reading a sparse matrix from a text file.

In [ ]:
# Create an empty lil matrix.
mtx = sparse.lil_matrix((4, 5))

In [ ]:
# Assign some of the indices, i.e., changing the sparsity.
mtx[:2, [1, 3]] = np.array([[1, 2], [3, 4]])

In [ ]:
mtx.toarray()

In [ ]:
# Read some of the indices.
mtx[:2].toarray()

### 2.2 Coordinate format (COO)

A COO matrix is constructed from three lists:
* a list of column indices,
* a list of row indices,
* a list of values,
where each element of those lists represents a non-zero element in the resulting sparse matrix.

This format is well-adapted to build a sparse adjacency matrix from an edge list.

In [ ]:
row = np.array([0, 3, 1, 0])  # row coordinates
col = np.array([0, 3, 1, 2])  # column coordinates
data = np.array([4, 5, 7, 9])  # values

mtx = sparse.coo_matrix((data, (row, col)), shape=(4, 4))

In [ ]:
mtx.toarray()

Advantages:
* Fast element-wise operations.
* Fast conversion to other sparse formats.

In [ ]:
# Element-wise power.
mtx.power(0.5).toarray()

In [ ]:
mtx_csr = mtx.tocsr()

Disadvantages:
* Indexing is not possible. (Use LIL instead!)
* Slow at arithmetic operations. (Use CSR, CSC instead!)

**Exercise:** Can you construct the sparse adjacency matrix in `COO` and `LIL` formats for a network given by the following edge list ?

In [ ]:
edges = pd.DataFrame(
    {"node_1": [1,1,1,2,3,3,3],
     "node_2": [3,4,5,6,4,5,6],
     "weights": [0.6,0.5,0.7,0.1,0.6,0.1,0.9]
    })
edges

### 2.3 Compressed sparse row & column formats (CSR & CSC)

In [ ]:
# Get the data array
mtx_csr.data

`CSR` is row oriented:
* efficient row slicing
* fast matrix vector products, the right multiplication `CSR * v`

In [ ]:
# Get array of column indices for CSR.
mtx_csr.indices

In [ ]:
# Matrix-vector product from the right.
v = np.array([1, 1, 1, 1])
mtx_csr.dot(v)

`CSC` is column oriented:
* efficient column slicing
* fast matrix vector products, the left multiplication `v * CSC`

In [ ]:
mtx_csc = mtx.tocsc()
# Get array of row indices for CSC
mtx_csc.indices

In [ ]:
# vectro-matrix product
v * mtx_csc

Efficient arithmetic operations `CSC + CSC`, `CSR * CSR`, etc.

In [ ]:
# Matrix-Matrix product (* is elementwise product on Numpy!)
prod = mtx_csc * mtx_csc
prod.toarray()

In [ ]:
prod = mtx_csr @ mtx_csr # @ is matrix product both on numpy and scipy!
prod.toarray()

You can read more about sparse matrix storage schemes [on Wikipedia](https://en.wikipedia.org/wiki/Sparse_matrix#Storing_a_sparse_matrix).

## 3. Linear agebra on sparse matrices

### 3.1 Some basic operations

In [ ]:
# sparse matrix from diagonals
A = sparse.spdiags(np.array([[1,2,3,4], [1,2,3,4], [1,2,3,4]]), [-1,0,2], 4, 4)
A.toarray()

**Inversion of a sparse matrix**

In [ ]:
A = A.tocsc()  # Convert it to CSC matrix for efficiency.
Ainv = sparse.linalg.inv(A)
Ainv.toarray()

In [ ]:
sparse.linalg.norm(A)  # Default to Frobenius norm.

**Solve $A x = b$**

In [ ]:
b = np.array([1, 1, 1, 1])
x = sparse.linalg.spsolve(A, b)
x

### 3.2 Eigenvalue decomposition

For the full eigendecomposition of an array, you can use the functions provided by Numpy:
* `numpy.linalg.eig`
* `numpy.linalg.eigvals`
* `numpy.linalg.eigh`
* `numpy.linalg.eighvals`

Scipy presents more functionality (read [here](https://www.scipy.org/scipylib/faq.html#why-both-numpy-linalg-and-scipy-linalg-what-s-the-difference)) such as solving generalized eigenvalue problem, you can use the functions from Scipy:
* `scipy.linalg.eig`
* `scipy.linalg.eigvals`
* `scipy.linalg.eigh`
* `scipy.linalg.eighvals`

In [ ]:
linalg.eigvals(A.toarray())

Decomposition of an Hermitian matrix:

In [ ]:
A = np.array([[1, -2j], [2j, 5]])
linalg.eigvalsh(A)

However, for quickly finding a few eigenvalues of a large sparse matrix, you should use the corresponding functions from the [sparse module](https://docs.scipy.org/doc/scipy/reference/tutorial/arpack.html):

* `scipy.sparse.eigs`
* `scipy.sparse.eigsh`

In [ ]:
dummy = sparse.random(30, 30, density=0.01)
evals, evecs = sparse.linalg.eigs(dummy, k=5, which='SM')
evals